In [1]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd


url_wikipedia = 'https://ru.wikipedia.org'
url_es = 'https://ru.wikipedia.org/wiki/Категория:Города_по_субъектам_Российской_Федерации'
url_es_q  = urllib.parse.quote(url_es, safe='/:')
print(url_es_q)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8


In [2]:
es_bs = BeautifulSoup(urllib.request.urlopen(url_es_q), 'lxml')

In [3]:
all_es = []
for i in range(30):
    # load links:
    bef = len(all_es)
    for links in es_bs.find_all('div', class_='mw-category-group'):
        for l in links.find_all('a'):
            all_es.append((l['title'], url_wikipedia+l['href']))

In [4]:
all_es

[('Категория:Категории городов по субъектам Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8'),
 ('Категория:Города Крыма',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%9A%D1%80%D1%8B%D0%BC%D0%B0'),
 ('Категория:Категории по городам субъектов Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%B0%D0%BC_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%

In [115]:

all_es_prop = []

for i, (n, lnk) in enumerate(all_es[:30]):
    es = BeautifulSoup(urllib.request.urlopen(lnk), 'lxml')
    es_pr = {}
    city_info = {}

    all_cities_in_group = []
    mw_pages = es.find("div", {"id": "mw-pages"})
    group_cities_links = []
    saved_th = []
    cities_rows = []

    cities = []

    df_parsed = pd.DataFrame()

    if mw_pages is not None:
        group_cities = mw_pages.find_all("a")
        for each_a in group_cities:
            if "Городские" in each_a["title"]:
                group_cities_links.append(f"https://ru.wikipedia.org{each_a['href']}")
                all_cities_in_group.append(each_a["title"])
                #print(each_a["href"], each_a["title"])
        #print(group_cities)
        #all_cities_in_group.append(group_cities["title"])
        #print(f"https://ru.wikipedia.org{group_cities['href']}")
        #group_cities_links.append(f"https://ru.wikipedia.org{group_cities['href']}")

    for link in group_cities_links:
        cs = BeautifulSoup(urllib.request.urlopen(link), 'lxml')
        cities_table = cs.find("table", class_="standard")
        #print(link)
        if cities_table is not None:
            for row in cities_table.find_all("tr"):
                th = row.find_all("th")
                #print("123")
                #print(th.text)
                if th:
                    #pass
                    #print(f"Th length: {len(th)}")
                    for i in range(len(th)):

                        #print(th[i].text.strip(), end=' || ')
                        saved_th.append(th[i].text.strip())
                        #df_parsed[th[i].text.strip()] = 0
                td = row.find_all('td')
                #print(td.text)
                if td:
                    #print("123")
                    for i in range(len(td)):
                        a_img = td[i].find_all("a", class_="image")

                        if a_img:
                            for each_img in a_img:
                                img_src = each_img.find_all("img")
                                if img_src:
                                    for each_src in img_src:

                                        image_link = each_src["src"]

                                        # print(f"Th[i]: {i}")
                                        # print(f"Th now: {saved_th}")
                                        # print(f"Th length now: {len(saved_th)}")
                                        # print(f"Image link in: {saved_th[i]}")

                        #city_info[saved_th[i]] = td[i].text.strip()
                        #es_pr[td[i].text.strip() if saved_th[i] == "№" else len(es_pr)] = {saved_th[i]: td[i].text.strip()}

                        es_pr[saved_th[i]] = td[i].text.strip()
                        if i == len(td)- 1:
                            #print(es_pr)
                            es_pr["Область"] = n.split(":")[1]
                            cities.append(es_pr)
                            #print(pd.json_normalize(es_pr))

                            es_pr = {}
                        #df_parsed[saved_th[i]] = td[i].text.strip()
                        #print(td[i].text.strip(), "----", saved_th[i])
                    #es_pr[td[i].text.strip()] = city_info
                    #print(cities)
                    #print("123")
    if len(cities) > 0:
        print(cities)
        #print(es_pr)
        all_es_prop.append({"Область": n.split(":")[1], "Города": cities})
        df_parsed[n] = cities
        #print('{}: {}, Название города: {}'.format(i, n, city_name))


[{'№': '1', 'название': 'Алупка', 'район[5] / город   республиканского значения[6]': 'Ялта', 'население(чел.)': '↗9063[4]', 'основание / первое упоминание': '960', 'статус города': '1938', 'герб': '', 'Область': 'Города Крыма'}, {'№': '2', 'название': 'Алушта', 'район[5] / город   республиканского значения[6]': 'Алушта', 'население(чел.)': '↗31\xa0364[4]', 'основание / первое упоминание': 'VI в.', 'статус города': '1902', 'герб': '', 'Область': 'Города Крыма'}, {'№': '3', 'название': 'Армянск', 'район[5] / город   республиканского значения[6]': 'Армянск', 'население(чел.)': '↘20\xa0692[4]', 'основание / первое упоминание': '1736', 'статус города': '1993', 'герб': '', 'Область': 'Города Крыма'}, {'№': '4', 'название': 'Бахчисарай', 'район[5] / город   республиканского значения[6]': 'Бахчисарайский район', 'население(чел.)': '↗28\xa0609[4]', 'основание / первое упоминание': '1389', 'статус города': '', 'герб': '', 'Область': 'Города Крыма'}, {'№': '5', 'название': 'Белогорск', 'район[5] 

[{'№': '1', 'город': 'Архангельск', 'район / город областного значения[7]': 'Архангельск', 'муниципальный район/ муниципальный округ/городской округ[8]': 'Архангельск, ГО', 'население,чел.': '↘301\xa0199[4]', 'основание / первое упоминание': '1584', 'статус города': '1584', 'герб': '', 'прежние названия': 'до 1613 — Новохолмогоры', 'Область': 'Города Архангельской области'}, {'№': '2', 'город': 'Вельск', 'район / город областного значения[7]': 'Вельский район', 'муниципальный район/ муниципальный округ/городской округ[8]': 'Вельский мун.район', 'население,чел.': '↘22\xa0217[6]', 'основание / первое упоминание': '1137', 'статус города': '1780', 'герб': '', 'прежние названия': '', 'Область': 'Города Архангельской области'}, {'№': '3', 'город': 'Каргополь', 'район / город областного значения[7]': 'Каргопольский район', 'муниципальный район/ муниципальный округ/городской округ[8]': 'Каргопольский мун.округ', 'население,чел.': '↗10\xa0025[6]', 'основание / первое упоминание': '1380', 'стату

[{'№': '1', 'название': 'Алексеевка', 'район / город (вне района)[6]': 'Алексеевский район', 'мун.район/городской округ[7]': 'Алексеевский городской округ', 'население(чел.)': '↘36\xa0578[5]', 'основание / первое упоминание': '1860', 'статус города': '1926', 'герб': '', 'прежние названия': '', 'Область': 'Города Белгородской области'}, {'№': '2', 'название': 'Белгород', 'район / город (вне района)[6]': 'город Белгород', 'мун.район/городской округ[7]': 'город Белгород, городской округ', 'население(чел.)': '↘339\xa0978[5]', 'основание / первое упоминание': '1596', 'статус города': '1596', 'герб': '', 'прежние названия': '', 'Область': 'Города Белгородской области'}, {'№': '3', 'название': 'Бирюч', 'район / город (вне района)[6]': 'Красногвардейский район', 'мун.район/городской округ[7]': 'Красногвардейский мун.район', 'население(чел.)': '↘7114[5]', 'основание / первое упоминание': '1705', 'статус города': '2005', 'герб': '', 'прежние названия': 'Будённый (1919—1958),Красногвардейское (19

[{'№': '1', 'название': 'Бабушкин', 'район/ город республиканского значения': 'Кабанский район', 'население(чел.)': '↘4368[3]', 'статусгорода с': '1902 года', 'Область': 'Города Бурятии'}, {'№': '2', 'название': 'Гусиноозёрск', 'район/ город республиканского значения': 'Селенгинский район', 'население(чел.)': '↗24\xa0451[3]', 'статусгорода с': '1953 года', 'Область': 'Города Бурятии'}, {'№': '3', 'название': 'Закаменск', 'район/ город республиканского значения': 'Закаменский район', 'население(чел.)': '↗11\xa0365[3]', 'статусгорода с': '1944 года', 'Область': 'Города Бурятии'}, {'№': '4', 'название': 'Кяхта', 'район/ город республиканского значения': 'Кяхтинский район', 'население(чел.)': '↘17\xa0877[3]', 'статусгорода с': '1805 года', 'Область': 'Города Бурятии'}, {'№': '5', 'название': 'Северобайкальск', 'район/ город республиканского значения': 'Северобайкальск, город', 'население(чел.)': '↗24\xa0233[3]', 'статусгорода с': '1980 года', 'Область': 'Города Бурятии'}, {'№': '6', 'назва

[{'№': '1', 'название': 'Бабаево', 'район / город  областного значения[8]': 'Бабаевский район', 'мун.район  / городской округ[9]': 'Бабаевский район', 'население(чел.)': '↗11\xa0416[4]', 'основание / первое упоминание': '1902', 'статус города': '1925', 'герб': '', 'прежние названия': '', 'Область': 'Города Вологодской области'}, {'№': '2', 'название': 'Белозерск', 'район / город  областного значения[8]': 'Белозерский район', 'мун.район  / городской округ[9]': 'Белозерский район', 'население(чел.)': '↘8464[4]', 'основание / первое упоминание': '862', 'статус города': '862', 'герб': '', 'прежние названия': 'Белоозеро', 'Область': 'Города Вологодской области'}, {'№': '3', 'название': 'Великий Устюг', 'район / город  областного значения[8]': 'Великий Устюг[5]', 'мун.район  / городской округ[9]': 'Великоустюгский район', 'население(чел.)': '↗31\xa0078[4]', 'основание / первое упоминание': '1147', 'статус города': '1147', 'герб': '', 'прежние названия': 'Устюг', 'Область': 'Города Вологодско

[{'№': '1', 'название': 'Буйнакск', 'город республиканского значения (городской округ)': 'Буйнакск', 'население(чел.)': '↗67\xa0091[4]', 'площадь(км²)': '20,95', 'основан': '1834', 'статусгорода': '1866', 'герб': '', 'прежнее название': 'до 1922 — Темир-Хан-Шура', 'Область': 'Города Дагестана'}, {'№': '2', 'название': 'Дагестанские Огни', 'город республиканского значения (городской округ)': 'Дагестанские Огни', 'население(чел.)': '↗30\xa0428[5]', 'площадь(км²)': '9,27', 'основан': '1914', 'статусгорода': '1990', 'герб': '', 'прежнее название': '', 'Область': 'Города Дагестана'}, {'№': '3', 'название': 'Дербент', 'город республиканского значения (городской округ)': 'Дербент', 'население(чел.)': '↗127\xa0518[6]', 'площадь(км²)': '69,63', 'основан': '438', 'статусгорода': '1840', 'герб': '', 'прежнее название': 'Баб-аль-абваб  Дарбанд Чалл Чалли', 'Область': 'Города Дагестана'}, {'№': '4', 'название': 'Избербаш', 'город республиканского значения (городской округ)': 'Избербаш', 'население(

[{'№': '1', 'название': 'Карабулак', 'город республиканского значения[4]': 'город Карабулак', 'население(чел.)': '↗44\xa0731[2]', 'основание первое упоминание': '1859', 'статус города': '1995', 'герб': '', 'прежние названия': 'Карабулакская (до 1962)', 'Область': 'Города Ингушетии'}, {'№': '2', 'название': 'Магас', 'город республиканского значения[4]': 'город Магас', 'население(чел.)': '↗15\xa0271[3]', 'основание первое упоминание': '1995', 'статус города': '2000', 'герб': '', 'прежние названия': '', 'Область': 'Города Ингушетии'}, {'№': '3', 'название': 'Малгобек', 'город республиканского значения[4]': 'город Малгобек', 'население(чел.)': '↗39\xa0794[2]', 'основание первое упоминание': '1934', 'статус города': '1939', 'герб': '', 'прежние названия': 'Вознесенское (до 1934)', 'Область': 'Города Ингушетии'}, {'№': '4', 'название': 'Назрань', 'город республиканского значения[4]': 'город Назрань', 'население(чел.)': '↗125\xa0297[2]', 'основание первое упоминание': '1781', 'статус города':

[{'№': '1', 'название': 'Багратионовск', 'годоснования': '1336', 'прежние названия': 'Прёйсиш-Эйлау', 'площадь  (км²)': '8', 'население(чел.)': '↗6401[4]', 'район  / город областного значения[6]': 'Багратионовский район', 'городской/муниципальный округ[7]': 'Багратионовский муниципальный округ', 'Область': 'Города Калининградской области'}, {'№': '2', 'название': 'Балтийск', 'годоснования': '1626', 'прежние названия': 'Пиллау', 'площадь  (км²)': '49', 'население(чел.)': '↗33\xa0946[4]', 'район  / город областного значения[6]': 'Балтийск', 'городской/муниципальный округ[7]': 'Балтийский городской округ', 'Область': 'Города Калининградской области'}, {'№': '3', 'название': 'Гвардейск', 'годоснования': '1255', 'прежние названия': 'Тапиау', 'площадь  (км²)': '12', 'население(чел.)': '↗13\xa0353[4]', 'район  / город областного значения[6]': 'Гвардейский район', 'городской/муниципальный округ[7]': 'Гвардейский муниципальный округ', 'Область': 'Города Калининградской области'}, {'№': '4', 'на

[{'№': '1', 'название': 'Вилючинск', 'район/город[3]': 'Вилючинск (ЗАТО)', 'мун.район  / городской округ[4]': 'Вилючинский городской округ (ЗАТО)', 'население(чел.)': '↘21\xa0774[2]', 'основание / первое упоминание': '1968', 'статус города': '1968', 'герб': '', 'Область': 'Города Камчатского края'}, {'№': '2', 'название': 'Елизово', 'район/город[3]': 'Елизовский район', 'мун.район  / городской округ[4]': 'Елизовский муниципальный район', 'население(чел.)': '↘36\xa0240[2]', 'основание / первое упоминание': '1848', 'статус города': '1975', 'герб': '', 'Область': 'Города Камчатского края'}, {'№': '3', 'название': 'Петропавловск-Камчатский', 'район/город[3]': 'Петропавловск-Камчатский', 'мун.район  / городской округ[4]': 'Петропавловск-Камчатский городской округ', 'население(чел.)': '↘164\xa0900[2]', 'основание / первое упоминание': '1740', 'статус города': '1812', 'герб': '', 'Область': 'Города Камчатского края'}]
[{'№': '1', 'название': 'Карачаевск', 'район/ город республиканского значен

In [78]:
pd.json_normalize(all_es_prop[0]["Города"]).set_index("Область")

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб
Область,,,,,,,
Города Крыма,1,Алупка,Ялта,↗9063[4],960,1938,
Города Крыма,2,Алушта,Алушта,↗31 364[4],VI в.,1902,
Города Крыма,3,Армянск,Армянск,↘20 692[4],1736,1993,
Города Крыма,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,
Города Крыма,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,
Города Крыма,6,Джанкой,Джанкой,↘37 014[4],1855,1917,
Города Крыма,7,Евпатория,Евпатория,↘107 877[4],497 до н. э.,,
Города Крыма,8,Керчь,Керчь,↗154 621[4],VII в. до н. э.,,
Города Крыма,9,Красноперекопск,Красноперекопск,↗25 569[4],1932,1966,


In [139]:
pd.json_normalize(all_es_prop)

,Область,Города
0,Города Крыма,"[{'№': '1', 'название': 'Алупка', 'район[5] / ..."
1,Города Адыгеи,"[{'№': '1', 'название': 'Адыгейск', 'город рес..."
2,Города Республики Алтай,"[{'№': '1', 'название': 'Горно-Алтайск', 'горо..."
3,Города Алтайского края,"[{'№': '1', 'название': 'Алейск', 'город краев..."
4,Города Амурской области,"[{'№': '1', 'название': 'Белогорск', 'район / ..."
5,Города Архангельской области,"[{'№': '1', 'город': 'Архангельск', 'район / г..."
6,Города Астраханской области,"[{'№': '1', 'название': 'Астрахань', 'район / ..."
7,Города Башкортостана,"[{'№': '1', 'Герб': '', 'Название': 'Агидельба..."
8,Города Белгородской области,"[{'№': '1', 'название': 'Алексеевка', 'район /..."
9,Города Брянской области,"[{'№': '1', 'название': 'Брянск', 'район/город..."


In [141]:
print(len(all_es_prop))

28


In [194]:

all_df = pd.json_normalize(all_es_prop[0]["Города"])


for i in range(len(all_es_prop) - 1):
    cities = pd.json_normalize(all_es_prop[i+1]["Города"])
    all_df = pd.concat([all_df, cities], names=["123"])
    
all_df

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб,Область,город республиканского значения (городской округ),основан,...,площадь (км²),район / город областного значения[6],городской/муниципальный округ[7],район[4] / город республиканского значения[5],основание /первое упоминание,муниципальный район / городской округ,район/город[3],мун.район / городской округ[4],район / город республиканского значения[6],мун. район / городской округ[7]
0,1,Алупка,Ялта,↗9063[4],960,1938,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Алушта,Алушта,↗31 364[4],VI в.,1902,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Армянск,Армянск,↘20 692[4],1736,1993,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Питкяранта(карел. Pitkyrandu),NaN,↘9818[4],NaN,1940,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1499,NaN,NaN,NaN,Питкярантский район,Питкярантский мун.район
9,10,Пудож(карел. Puudoži),NaN,↘8387[4],NaN,1785,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1382,NaN,NaN,NaN,Пудожский район,Пудожский мун.район
10,11,Сегежа(карел. Segeža),NaN,↘25 334[4],NaN,1943,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1914,NaN,NaN,NaN,Сегежский район,Сегежский мун.район
11,12,Сортавала(карел. Sortavala),NaN,↘18 600[4],NaN,1783,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1468,NaN,NaN,NaN,Сортавала,Сортавальский мун.район


In [135]:
#Сколько городов в каждой области
all_df.groupby("Область")["название"].count()

Область
Города Адыгеи                           2
Города Алтайского края                 12
Города Амурской области                10
Города Архангельской области            0
Города Астраханской области             6
Города Башкортостана                    0
Города Белгородской области            11
Города Брянской области                16
Города Бурятии                          6
Города Владимирской области            23
Города Волгоградской области            0
Города Вологодской области             15
Города Воронежской области             15
Города Дагестана                       10
Города Еврейской автономной области     0
Города Забайкальского края              0
Города Ивановской области              17
Города Ингушетии                        5
Города Иркутской области                0
Города Кабардино-Балкарии               0
Города Калининградской области         22
Города Калмыкии                         3
Города Калужской области               22
Города Камчатского края   

In [146]:
all_df.count()

№                                                  329
название                                           232
район[5] / город   республиканского значения[6]     16
население(чел.)                                    240
основание / первое упоминание                      191
                                                  ... 
муниципальный район  / городской округ              22
район/город[3]                                       3
мун.район  / городской округ[4]                      3
район / город  республиканского значения[6]         13
мун. район /  городской округ[7]                    13
Length: 69, dtype: int64

In [147]:
all_df.dtypes

№                                                  object
название                                           object
район[5] / город   республиканского значения[6]    object
население(чел.)                                    object
основание / первое упоминание                      object
                                                    ...  
муниципальный район  / городской округ             object
район/город[3]                                     object
мун.район  / городской округ[4]                    object
район / город  республиканского значения[6]        object
мун. район /  городской округ[7]                   object
Length: 69, dtype: object

In [195]:
import re

all_df["население(чел.)"] = all_df["население(чел.)"].astype(str).apply(lambda x: "".join(re.findall('[0-9]+', x)[:-1]))

In [196]:
all_df["население(чел.)"] = pd.to_numeric(all_df["население(чел.)"], downcast="integer")

In [199]:
all_df

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб,Область,город республиканского значения (городской округ),основан,...,площадь (км²),район / город областного значения[6],городской/муниципальный округ[7],район[4] / город республиканского значения[5],основание /первое упоминание,муниципальный район / городской округ,район/город[3],мун.район / городской округ[4],район / город республиканского значения[6],мун. район / городской округ[7]
0,1,Алупка,Ялта,9063.0,960,1938,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Алушта,Алушта,31364.0,VI в.,1902,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Армянск,Армянск,20692.0,1736,1993,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Бахчисарай,Бахчисарайский район,28609.0,1389,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Белогорск,Белогорский район,17445.0,XIII в.,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Питкяранта(карел. Pitkyrandu),NaN,9818.0,NaN,1940,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1499,NaN,NaN,NaN,Питкярантский район,Питкярантский мун.район
9,10,Пудож(карел. Puudoži),NaN,8387.0,NaN,1785,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1382,NaN,NaN,NaN,Пудожский район,Пудожский мун.район
10,11,Сегежа(карел. Segeža),NaN,25334.0,NaN,1943,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1914,NaN,NaN,NaN,Сегежский район,Сегежский мун.район
11,12,Сортавала(карел. Sortavala),NaN,18600.0,NaN,1783,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1468,NaN,NaN,NaN,Сортавала,Сортавальский мун.район


In [197]:
all_df.groupby("Область")["население(чел.)"].describe()

,count,mean,std,min,25%,50%,75%,max
Область,,,,,,,,
Города Адыгеи,2.0,78147.500000,92259.757275,12910.0,45528.75,78147.5,110766.25,143385.0
Города Алтайского края,12.0,103476.666667,175824.227535,10187.0,17236.25,34525.5,90830.75,630877.0
Города Амурской области,10.0,45558.400000,71162.268589,7057.0,11160.50,17951.0,43544.00,241437.0
Города Архангельской области,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Города Астраханской области,6.0,96944.000000,185708.115688,11104.0,16744.00,21571.0,32883.25,475629.0
Города Башкортостана,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Города Белгородской области,11.0,74293.636364,107842.562612,5629.0,12736.50,33032.0,62452.50,339978.0
Города Брянской области,16.0,41792.562500,91078.376294,5323.0,12042.50,16565.0,23237.50,379152.0
Города Бурятии,6.0,86643.166667,172089.406247,4368.0,12993.00,21055.0,24396.50,437565.0
